In [89]:
!pip install --upgrade --quiet langchain langchain-community  langchain-experimental tiktoken yfiles-jupyter-graphs

In [90]:
from typing import List, Tuple,Optional

In [91]:
from google.colab import userdata
groq_api=userdata.get('groq_api')
mistral_api_key=userdata.get('Mistral_api_key')
hf_token=userdata.get('HF_TOKEN')

In [92]:
NEO4J_URI="neo4j+s://9c018341.databases.neo4j.io"
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD="z6btw246zOd-2ItOlLm9meHt9bUYZWLBaP1dqDn_znM"
AURA_INSTANCEID="9c018341"
AURA_INSTANCENAME="Instance01"

In [93]:
import os
os.environ["groq_api"]=groq_api
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [94]:
from langchain_community.graphs import Neo4jGraph

In [95]:
!pip install neo4j

In [96]:
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

In [97]:
!pip install wikipedia

In [98]:
from langchain.document_loaders import WikipediaLoader
raw_document=WikipediaLoader(query="Elizabeth I").load()

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [99]:
len(raw_document)

23

In [100]:
from langchain.text_splitter import TokenTextSplitter
text_splitter=TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents=text_splitter.split_documents(raw_document[:3])

In [101]:
documents[0]

Document(metadata={'title': 'Elizabeth I', 'summary': 'Elizabeth I (7 September 1533 – 24 March 1603) was Queen of England and Ireland from 17 November 1558 until her death in 1603. She was the last monarch of the House of Tudor.\nElizabeth was the only surviving child of Henry VIII and his second wife, Anne Boleyn. When Elizabeth was two years old, her parents\' marriage was annulled, her mother was executed, and Elizabeth was declared illegitimate. Henry restored her to the line of succession when she was 10, via the Third Succession Act 1543. After Henry\'s death in 1547, Elizabeth\'s younger half-brother Edward VI ruled until his own death in 1553, bequeathing the crown to a Protestant cousin, Lady Jane Grey, and ignoring the claims of his two half-sisters, the Catholic Mary and the younger Elizabeth, in spite of statutes to the contrary. Edward\'s will was set aside within weeks of his death and Mary became queen, deposing and executing Jane. During Mary\'s reign, Elizabeth was im

In [102]:
!pip install langchain_groq

In [103]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api,model_name="llama-3.1-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7a32dcce4520>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7a32dcce9060>, model_name='llama-3.1-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [104]:
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [105]:
llm_transformer=LLMGraphTransformer(llm=llm)

In [106]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [107]:
graph_documents

[GraphDocument(nodes=[Node(id='Elizabeth I', type='Person', properties={}), Node(id='Henry Viii', type='Person', properties={}), Node(id='Anne Boleyn', type='Person', properties={}), Node(id='Edward Vi', type='Person', properties={}), Node(id='Lady Jane Grey', type='Person', properties={}), Node(id='Mary', type='Person', properties={}), Node(id='William Cecil', type='Person', properties={}), Node(id='James Vi Of Scotland', type='Person', properties={}), Node(id='Mary, Queen Of Scots', type='Person', properties={}), Node(id='Sir Francis Walsingham', type='Person', properties={}), Node(id='House Of Tudor', type='Organization', properties={}), Node(id='Church Of England', type='Organization', properties={}), Node(id='England', type='Location', properties={}), Node(id='Ireland', type='Location', properties={}), Node(id='Scotland', type='Location', properties={}), Node(id='France', type='Location', properties={}), Node(id='Spain', type='Location', properties={}), Node(id='Netherlands', type

In [108]:
graph.add_graph_documents(
    graph_documents=graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [109]:
default_cypher="MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [110]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [111]:
def showGraph(cypher:str=default_cypher):
  driver=GraphDatabase.driver(
      uri=os.environ["NEO4J_URI"],
      auth=(os.environ["NEO4J_USERNAME"],
      os.environ["NEO4J_PASSWORD"])
  )

  session=driver.session()
  widget=GraphWidget(graph=session.run(cypher).graph())
  widget.node_label_mapping='id'
  display(widget)
  return widget

In [112]:
from google.colab import output
output.disable_custom_widget_manager()

## To visualize the graph in local

In [113]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [114]:
!pip install -U langchain_mistralai

In [115]:
from langchain_mistralai import MistralAIEmbeddings

In [116]:
!pip install huggingface_hub transformers

In [117]:
embedding = MistralAIEmbeddings(api_key=mistral_api_key,model="mistral-embed")

/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:175: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [121]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

In [125]:
model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}

In [126]:
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [127]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [128]:
from langchain_core.pydantic_v1 import BaseModel,Field

class Entities(BaseModel):
  names:List[str]=Field(
      ...,
      description="All the person, organization, or business entities that"
      "appear in the text",
  )

In [129]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [130]:
prompt=ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
            ),
        (
            "human",
            "Use the given format to extract information from the folllowing"
            "input :{question}",
        ),
    ]
)

In [131]:
entity_chain=prompt|llm.with_structured_output(Entities)

In [132]:
entity_chain.invoke({"question":"Where was the Amelia Earhart born?"}).names

['Amelia Earhart']

In [133]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [134]:
def generate_full_text_query(input:str)-> str:
  full_text_query=""
  words=[el for el in remove_lucene_chars(input).split(" ") if el]
  for word in words[:-1]:
    full_text_query+=f"{word}~2 AND"
  full_text_query+=f"{words[-1]}~2"
  return full_text_query.strip()

In [142]:
def generate_query_string(entity: str) -> str:
    """Generate a simple search string"""
    return entity.replace('"', '\\"')

def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})

    for entity in entities.names:
        response = graph.query(
            """
            MATCH (node:Document)
            WHERE toLower(node.text) CONTAINS toLower($query)
            WITH node
            OPTIONAL MATCH (node)-[r:MENTIONS]->(neighbor)
            WITH node, collect(CASE
                WHEN neighbor IS NOT NULL
                THEN node.id + ' - ' + type(r) + ' -> ' + neighbor.id
                ELSE null
            END) as outgoing
            OPTIONAL MATCH (node)<-[r:MENTIONS]-(neighbor)
            WITH node, outgoing, collect(CASE
                WHEN neighbor IS NOT NULL
                THEN neighbor.id + ' - ' + type(r) + ' -> ' + node.id
                ELSE null
            END) as incoming
            RETURN [x IN outgoing + incoming WHERE x IS NOT NULL] as output
            LIMIT 50
            """,
            params={"query": generate_query_string(entity)}
        )

        # Process results
        for record in response:
            if record['output']:
                result += "\n".join(record['output']) + "\n"

    return result.strip()

In [144]:
result = structured_retriever("Who is Elizabeth I?")
result

"2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Elizabeth_I\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Henry_Viii\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Anne_Boleyn\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Edward_Vi\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Lady_Jane_Grey\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Mary\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> William_Cecil\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> James_Vi_Of_Scotland\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Mary_Queen_Of_Scots\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Sir_Francis_Walsingham\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> House_Of_Tudor\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Church_Of_England\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Elizabeth I\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Lady Jane Grey\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> William Cecil\n2912ff1bf15ba29f11702607f35da890 - MENTIONS -> Anne Boleyn\n29

In [145]:
def retriver(question:str):
  print(f"search query: {question}")
  structured_data=structured_retriever(question)
  unstructured_data=[el.page_content for el in vector_index.similarity_search(question)]
  final_data=f"""structured data:
{structured_data}
Unstructured data:
{"#Document ".join(unstructured_data)}"""
  return final_data

In [146]:
template="""Given the following conversation and a follow up question rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [147]:
CONDENSE_QUESTION_PROMPT=PromptTemplate.from_template(template)

In [148]:
def format_chat_history(chat_histiry: List[Tuple[str,str]])->List:
  buffer=[]
  for human, ai in chat_history:
    buffer.append(HumanMessage(content=human))
    buffer.append(AIMessage(content=ai))
  return buffer

In [160]:
from langchain.schema.runnable import RunnableBranch, RunnableLambda, RunnablePassthrough,RunnableParallel

In [155]:
from langchain.schema.output_parser import StrOutputParser

In [156]:
search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm
        | StrOutputParser()
    ),
    # Default branch if no chat history
    RunnableLambda(lambda x: x["question"])
)

In [157]:
template="""Answer the question based only on the following context:
{context}
Question: {question}
Answer:"""

In [158]:
prompt=ChatPromptTemplate.from_template(template)

In [163]:
chain=(
    RunnableParallel(
        {
            "context": search_query,
            "question":RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)


In [164]:
chain.invoke({"question":"Which house did Elizabeth I belong to?"})

'Tudor.'